# Spark assignment 1: Pairs
Find all the pairs of two consequent words where the first word is “narodnaya”. For each pair, count the number of occurrences in the Wikipedia dump. Print all the pairs with their count in a lexicographical order. Output format is “word_pair <tab> count”, for example:

red_apple 100500

crazy_zoo 42

Note that two words in a pair are concatenated with the underscore character, and the result is in the lowercase.

One motivation for counting these continuations is to get a better understanding of the language. Some words, like “the”, have a lot of continuations, while others, like “San”, have just a few (“San Francisco”, for example). One can build a language model with these statistics. If you are interested to learn more, search for “n-gram language model” in the Internet.

Dataset location: /data/wiki/en_articles_part

The result on the sample dataset:

narodnaya_gazeta 1

narodnaya_volya 9

If you want to deploy the environment on your own machine, please use bigdatateam/all-spark-with-data Docker container. New image: bigdatateam/hysh-full:py3-c1


In [1]:
from pyspark import SparkConf, SparkContext

try:
    sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local").set("spark.cores.max", "16"))
except:
    pass

import re

In [2]:
def parse_article(line):
    try:
        article_id, text = line.rstrip().split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

In [3]:
def search(words):
    results = []
    for word1, word2 in zip(words, words[1:]):
        results.append((word1.lower() + "_" + word2.lower(), 1))
    return results

In [5]:
results = (sc.textFile("/data/wiki/en_articles_part/articles-part", 16)
        .map(parse_article)
        .flatMap(search)
        .reduceByKey(lambda a, b : a + b)
        .filter(lambda value: value[0][:9] == "narodnaya")
       ).sortByKey().collect()

In [7]:
for key, count in results:
    print("%s\t%d" % (key, count))

narodnaya_gazeta	1
narodnaya_volya	9
